In [7]:
import pandas as pd
import ast
import numpy as np

In [2]:
out = pd.read_csv("debate_data.csv")
out.head()

,type,id,url,date,motion,score,score_upd,comp2,league,tournament,teams
0,debate,11360,https://statistiky.debatovani.cz/?page=debata&...,2025-12-13 17:00:00,As Victor Frankenstein we would accept the off...,vyhráli 3:0,vyhráli 3:0,Debate Cup XXIX.,"{'name': 'Debate League XXXI.', 'id': '47'}",{'name': 'Druhý turnaj Debate League 2025/2026...,"[{'team_name': 'Varáci', 'side': 'aff', 'speak..."
1,debate,11361,https://statistiky.debatovani.cz/?page=debata&...,2025-12-13 17:00:00,As Victor Frankenstein we would accept the off...,vyhráli 3:0,vyhráli 3:0,Debate Cup XXIX.,"{'name': 'Debate League XXXI.', 'id': '47'}",{'name': 'Druhý turnaj Debate League 2025/2026...,"[{'team_name': 'Los Debateros', 'side': 'aff',..."
2,debate,11362,https://statistiky.debatovani.cz/?page=debata&...,2025-12-13 17:00:00,As Victor Frankenstein we would accept the off...,vyhráli 3:0,vyhráli 3:0,Debate Cup XXIX.,"{'name': 'Debate League XXXI.', 'id': '47'}",{'name': 'Druhý turnaj Debate League 2025/2026...,"[{'team_name': 'Horáková a zemědělci', 'side':..."
3,debate,11363,https://statistiky.debatovani.cz/?page=debata&...,2025-12-13 17:00:00,As Victor Frankenstein we would accept the off...,vyhráli 3:0,vyhráli 3:0,Debate Cup XXIX.,"{'name': 'Debate League XXXI.', 'id': '47'}",{'name': 'Druhý turnaj Debate League 2025/2026...,"[{'team_name': 'Bonjour', 'side': 'aff', 'spea..."
4,debate,11364,https://statistiky.debatovani.cz/?page=debata&...,2025-12-13 17:00:00,As Victor Frankenstein we would accept the off...,vyhráli 3:0,vyhráli 3:0,Debate Cup XXIX.,"{'name': 'Debate League XXXI.', 'id': '47'}",{'name': 'Druhý turnaj Debate League 2025/2026...,"[{'team_name': 'Dobrodružný Klement jí kočku',..."


In [3]:
out["score"]

0     vyhráli 3:0
1     vyhráli 3:0
2     vyhráli 3:0
3     vyhráli 3:0
4     vyhráli 3:0
5     vyhráli 2:1
6     vyhráli 3:0
7     vyhráli 3:0
8     vyhráli 3:0
9     vyhráli 3:0
10    vyhráli 3:0
11    vyhráli 3:0
12    vyhráli 3:0
13    vyhráli 3:0
14    vyhráli 3:0
15    vyhráli 3:0
16    vyhráli 3:0
17    vyhráli 2:1
18    vyhráli 3:0
19    vyhráli 2:1
Name: score, dtype: object

In [4]:
out["score_upd"]

0     vyhráli 3:0
1     vyhráli 3:0
2     vyhráli 3:0
3     vyhráli 3:0
4     vyhráli 3:0
5     vyhráli 2:1
6     vyhráli 3:0
7     vyhráli 3:0
8     vyhráli 3:0
9     vyhráli 3:0
10    vyhráli 3:0
11    vyhráli 3:0
12    vyhráli 3:0
13    vyhráli 3:0
14    vyhráli 3:0
15    vyhráli 3:0
16    vyhráli 3:0
17    vyhráli 2:1
18    vyhráli 3:0
19    vyhráli 2:1
Name: score_upd, dtype: object

In [8]:
def process_debate_teams(df, team_column='teams'):
    def expand_row(row):
        try:
            teams_list = ast.literal_eval(row[team_column])
        except (ValueError, SyntaxError, TypeError):
            return pd.Series({})

        extracted_data = {}
        
        for team in teams_list:
            # side for prefix creation (aff_ or neg_)
            side = team.get('side', '').lower()
            if side not in ['aff', 'neg']:
                continue
            extracted_data[f'{side}_team_name'] = team.get('team_name')
            
            # speakers mapping (max is always 3 for team)
            speakers = team.get('speakers', [])
            for i in range(1, 4):
                if i-1 < len(speakers):
                    s = speakers[i-1]
                    extracted_data[f'{side}_speaker{i}'] = s.get('name')
                    extracted_data[f'{side}_speaker{i}_points'] = s.get('points')
                else:
                    # just a robustness helper if team had less than 3 
                    extracted_data[f'{side}_speaker{i}'] = None
                    extracted_data[f'{side}_speaker{i}_points'] = None
                    
        return pd.Series(extracted_data)
    expanded_info = df.apply(expand_row, axis=1)
    final_df = pd.concat([df.drop(columns=[team_column]), expanded_info], axis=1)
    return final_df
out2 = process_debate_teams(out)
out2.head()

,type,id,url,date,motion,score,score_upd,comp2,league,tournament,...,aff_speaker2_points,aff_speaker3,aff_speaker3_points,neg_team_name,neg_speaker1,neg_speaker1_points,neg_speaker2,neg_speaker2_points,neg_speaker3,neg_speaker3_points
0,debate,11360,https://statistiky.debatovani.cz/?page=debata&...,2025-12-13 17:00:00,As Victor Frankenstein we would accept the off...,vyhráli 3:0,vyhráli 3:0,Debate Cup XXIX.,"{'name': 'Debate League XXXI.', 'id': '47'}",{'name': 'Druhý turnaj Debate League 2025/2026...,...,87,Prokeš Patrik,83.0,Máme pravdu,Kurfürstová Eliška,73,Martochová Apolena,76,Bot Zoe Kaitlyn,76
1,debate,11361,https://statistiky.debatovani.cz/?page=debata&...,2025-12-13 17:00:00,As Victor Frankenstein we would accept the off...,vyhráli 3:0,vyhráli 3:0,Debate Cup XXIX.,"{'name': 'Debate League XXXI.', 'id': '47'}",{'name': 'Druhý turnaj Debate League 2025/2026...,...,76,Lehner Naomi Eva,74.0,Duševní katarze jímající kuřáka,Špinka Vít,71,Šnejdar Martin,69,Vokurka Václav,69
2,debate,11362,https://statistiky.debatovani.cz/?page=debata&...,2025-12-13 17:00:00,As Victor Frankenstein we would accept the off...,vyhráli 3:0,vyhráli 3:0,Debate Cup XXIX.,"{'name': 'Debate League XXXI.', 'id': '47'}",{'name': 'Druhý turnaj Debate League 2025/2026...,...,78,Požár Yurij,79.0,Kde jsou ty motyky,Šizlingová Johana,74,Železný Šimon,72,Mojžíšová Barbora,70
3,debate,11363,https://statistiky.debatovani.cz/?page=debata&...,2025-12-13 17:00:00,As Victor Frankenstein we would accept the off...,vyhráli 3:0,vyhráli 3:0,Debate Cup XXIX.,"{'name': 'Debate League XXXI.', 'id': '47'}",{'name': 'Druhý turnaj Debate League 2025/2026...,...,79,Šneider Libor,83.0,Memoriál D.H.,Wellartová Vivien Fay,79,Churý Tomáš,80,Juránek Prokop,78
4,debate,11364,https://statistiky.debatovani.cz/?page=debata&...,2025-12-13 17:00:00,As Victor Frankenstein we would accept the off...,vyhráli 3:0,vyhráli 3:0,Debate Cup XXIX.,"{'name': 'Debate League XXXI.', 'id': '47'}",{'name': 'Druhý turnaj Debate League 2025/2026...,...,80,Brávek Šimon,75.0,Cihla,Belatková Anna,77,Škorpilová Kateřina,79,Schmidová Jana,76


In [ ]:
out2

In [9]:
tt = pd.read_csv("debate_data.csv")
tt.head()

,type,id,url,date,motion,score,score_upd,judges,comp,league,tournament,teams
0,debate,11360,https://statistiky.debatovani.cz/?page=debata&...,2025-12-13 17:00:00,As Victor Frankenstein we would accept the off...,vyhráli 3:0,vyhráli 3:0,"[{'name': 'Soukup Václav', 'side': 'neg', 'sco...",Debate Cup XXIX.,"{'name': 'Debate League XXXI.', 'id': '47'}",{'name': 'Druhý turnaj Debate League 2025/2026...,"[{'team_name': 'Varáci', 'side': 'aff', 'speak..."
1,debate,11361,https://statistiky.debatovani.cz/?page=debata&...,2025-12-13 17:00:00,As Victor Frankenstein we would accept the off...,vyhráli 3:0,vyhráli 3:0,"[{'name': 'Bojko David', 'side': 'aff', 'score...",Debate Cup XXIX.,"{'name': 'Debate League XXXI.', 'id': '47'}",{'name': 'Druhý turnaj Debate League 2025/2026...,"[{'team_name': 'Los Debateros', 'side': 'aff',..."
2,debate,11362,https://statistiky.debatovani.cz/?page=debata&...,2025-12-13 17:00:00,As Victor Frankenstein we would accept the off...,vyhráli 3:0,vyhráli 3:0,"[{'name': 'Malík Marek', 'side': 'aff', 'score...",Debate Cup XXIX.,"{'name': 'Debate League XXXI.', 'id': '47'}",{'name': 'Druhý turnaj Debate League 2025/2026...,"[{'team_name': 'Horáková a zemědělci', 'side':..."
3,debate,11363,https://statistiky.debatovani.cz/?page=debata&...,2025-12-13 17:00:00,As Victor Frankenstein we would accept the off...,vyhráli 3:0,vyhráli 3:0,"[{'name': 'Lacinová Barbora', 'side': 'neg', '...",Debate Cup XXIX.,"{'name': 'Debate League XXXI.', 'id': '47'}",{'name': 'Druhý turnaj Debate League 2025/2026...,"[{'team_name': 'Bonjour', 'side': 'aff', 'spea..."
4,debate,11364,https://statistiky.debatovani.cz/?page=debata&...,2025-12-13 17:00:00,As Victor Frankenstein we would accept the off...,vyhráli 3:0,vyhráli 3:0,"[{'name': 'Long Lý Bao', 'side': 'aff', 'score...",Debate Cup XXIX.,"{'name': 'Debate League XXXI.', 'id': '47'}",{'name': 'Druhý turnaj Debate League 2025/2026...,"[{'team_name': 'Dobrodružný Klement jí kočku',..."


In [20]:
cc = pd.read_csv("debate_data.csv")
cc.head()

,type,id,date,comp,league,test_league_name,test_league_id,motion,score,judges,tournament,teams
0,debate,11360,2025-12-13 17:00:00,Debate Cup XXIX.,"{'name': 'Debate League XXXI.', 'id': '47'}",Debate League XXXI.,47.0,As Victor Frankenstein we would accept the off...,vyhráli 3:0,"[{'name': 'Soukup Václav', 'side': 'neg', 'sco...",{'name': 'Druhý turnaj Debate League 2025/2026...,"[{'team_name': 'Varáci', 'side': 'aff', 'speak..."
1,debate,11361,2025-12-13 17:00:00,Debate Cup XXIX.,"{'name': 'Debate League XXXI.', 'id': '47'}",Debate League XXXI.,47.0,As Victor Frankenstein we would accept the off...,vyhráli 3:0,"[{'name': 'Bojko David', 'side': 'aff', 'score...",{'name': 'Druhý turnaj Debate League 2025/2026...,"[{'team_name': 'Los Debateros', 'side': 'aff',..."
2,debate,11362,2025-12-13 17:00:00,Debate Cup XXIX.,"{'name': 'Debate League XXXI.', 'id': '47'}",Debate League XXXI.,47.0,As Victor Frankenstein we would accept the off...,vyhráli 3:0,"[{'name': 'Malík Marek', 'side': 'aff', 'score...",{'name': 'Druhý turnaj Debate League 2025/2026...,"[{'team_name': 'Horáková a zemědělci', 'side':..."
3,debate,11363,2025-12-13 17:00:00,Debate Cup XXIX.,"{'name': 'Debate League XXXI.', 'id': '47'}",Debate League XXXI.,47.0,As Victor Frankenstein we would accept the off...,vyhráli 3:0,"[{'name': 'Lacinová Barbora', 'side': 'neg', '...",{'name': 'Druhý turnaj Debate League 2025/2026...,"[{'team_name': 'Bonjour', 'side': 'aff', 'spea..."
4,debate,11364,2025-12-13 17:00:00,Debate Cup XXIX.,"{'name': 'Debate League XXXI.', 'id': '47'}",Debate League XXXI.,47.0,As Victor Frankenstein we would accept the off...,vyhráli 3:0,"[{'name': 'Long Lý Bao', 'side': 'aff', 'score...",{'name': 'Druhý turnaj Debate League 2025/2026...,"[{'team_name': 'Dobrodružný Klement jí kočku',..."


In [29]:
cc["tournament"]

0     {'name': 'Druhý turnaj Debate League 2025/2026...
1     {'name': 'Druhý turnaj Debate League 2025/2026...
2     {'name': 'Druhý turnaj Debate League 2025/2026...
3     {'name': 'Druhý turnaj Debate League 2025/2026...
4     {'name': 'Druhý turnaj Debate League 2025/2026...
5     {'name': 'Druhý turnaj Debate League 2025/2026...
6     {'name': 'Druhý turnaj Debate League 2025/2026...
7     {'name': 'Druhý turnaj Debate League 2025/2026...
8     {'name': 'Druhý turnaj Debate League 2025/2026...
9     {'name': 'Druhý turnaj Debate League 2025/2026...
10    {'name': 'Druhý turnaj Debate League 2025/2026...
11    {'name': 'Druhý turnaj Debate League 2025/2026...
12    {'name': 'Druhý turnaj Debate League 2025/2026...
13    {'name': 'Druhý turnaj Debate League 2025/2026...
14    {'name': 'Druhý turnaj Debate League 2025/2026...
15    {'name': 'Druhý turnaj Debate League 2025/2026...
16    {'name': 'Druhý turnaj Debate League 2025/2026...
17    {'name': 'Druhý turnaj Debate League 2025/

In [31]:
ccc["tournament_id"]

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
6    NaN
7    NaN
8    NaN
9    NaN
10   NaN
11   NaN
12   NaN
13   NaN
14   NaN
15   NaN
16   NaN
17   NaN
18   NaN
19   NaN
Name: tournament_id, dtype: float64

In [21]:
ccc = pd.read_csv("debate_data.csv")
ccc.head()

,type,id,date,comp,league_name,league_id,motion,score,judges_scoring,tournament_name,tournament_id,teams
0,debate,11360,2025-12-13 17:00:00,Debate Cup XXIX.,Debate League XXXI.,47.0,As Victor Frankenstein we would accept the off...,vyhráli 3:0,"[{'name': 'Soukup Václav', 'side': 'neg', 'sco...",Druhý turnaj Debate League 2025/2026,NaN,"[{'team_name': 'Varáci', 'side': 'aff', 'speak..."
1,debate,11361,2025-12-13 17:00:00,Debate Cup XXIX.,Debate League XXXI.,47.0,As Victor Frankenstein we would accept the off...,vyhráli 3:0,"[{'name': 'Bojko David', 'side': 'aff', 'score...",Druhý turnaj Debate League 2025/2026,NaN,"[{'team_name': 'Los Debateros', 'side': 'aff',..."
2,debate,11362,2025-12-13 17:00:00,Debate Cup XXIX.,Debate League XXXI.,47.0,As Victor Frankenstein we would accept the off...,vyhráli 3:0,"[{'name': 'Malík Marek', 'side': 'aff', 'score...",Druhý turnaj Debate League 2025/2026,NaN,"[{'team_name': 'Horáková a zemědělci', 'side':..."
3,debate,11363,2025-12-13 17:00:00,Debate Cup XXIX.,Debate League XXXI.,47.0,As Victor Frankenstein we would accept the off...,vyhráli 3:0,"[{'name': 'Lacinová Barbora', 'side': 'neg', '...",Druhý turnaj Debate League 2025/2026,NaN,"[{'team_name': 'Bonjour', 'side': 'aff', 'spea..."
4,debate,11364,2025-12-13 17:00:00,Debate Cup XXIX.,Debate League XXXI.,47.0,As Victor Frankenstein we would accept the off...,vyhráli 3:0,"[{'name': 'Long Lý Bao', 'side': 'aff', 'score...",Druhý turnaj Debate League 2025/2026,NaN,"[{'team_name': 'Dobrodružný Klement jí kočku',..."


In [32]:
cccc = pd.read_csv("debate_data.csv")
cccc.head()

,type,id,date,comp,league_name,league_id,motion,tournament_name,tournament_id,score,judges_scoring,teams
0,debate,11360,2025-12-13 17:00:00,Debate Cup XXIX.,Debate League XXXI.,47.0,As Victor Frankenstein we would accept the off...,Druhý turnaj Debate League 2025/2026,320.0,vyhráli 3:0,"[{'name': 'Soukup Václav', 'side': 'neg', 'sco...","[{'team_name': 'Varáci', 'side': 'aff', 'speak..."
1,debate,11361,2025-12-13 17:00:00,Debate Cup XXIX.,Debate League XXXI.,47.0,As Victor Frankenstein we would accept the off...,Druhý turnaj Debate League 2025/2026,320.0,vyhráli 3:0,"[{'name': 'Bojko David', 'side': 'aff', 'score...","[{'team_name': 'Los Debateros', 'side': 'aff',..."
2,debate,11362,2025-12-13 17:00:00,Debate Cup XXIX.,Debate League XXXI.,47.0,As Victor Frankenstein we would accept the off...,Druhý turnaj Debate League 2025/2026,320.0,vyhráli 3:0,"[{'name': 'Malík Marek', 'side': 'aff', 'score...","[{'team_name': 'Horáková a zemědělci', 'side':..."
3,debate,11363,2025-12-13 17:00:00,Debate Cup XXIX.,Debate League XXXI.,47.0,As Victor Frankenstein we would accept the off...,Druhý turnaj Debate League 2025/2026,320.0,vyhráli 3:0,"[{'name': 'Lacinová Barbora', 'side': 'neg', '...","[{'team_name': 'Bonjour', 'side': 'aff', 'spea..."
4,debate,11364,2025-12-13 17:00:00,Debate Cup XXIX.,Debate League XXXI.,47.0,As Victor Frankenstein we would accept the off...,Druhý turnaj Debate League 2025/2026,320.0,vyhráli 3:0,"[{'name': 'Long Lý Bao', 'side': 'aff', 'score...","[{'team_name': 'Dobrodružný Klement jí kočku',..."


In [36]:
ccc["teams"][5]

"[{'team_name': 'Pozor kluzká podlaha!', 'side': 'aff', 'speakers': [{'name': 'Ježák Adam', 'points': 71}, {'name': 'Foltová Nikol', 'points': 73}, {'name': 'Dostálová Barbora', 'points': 76}]}, {'team_name': 'deada$$', 'side': 'neg', 'speakers': [{'name': 'Fryčová Lucie', 'points': 76}, {'name': 'Luňáček Jan', 'points': 74}, {'name': 'Drdličko Dominik', 'points': 72}]}]"

In [17]:
tt["judges"][6]

nan

In [19]:
tt.iloc[6]

type                                                     debate
id                                                        11366
url           https://statistiky.debatovani.cz/?page=debata&...
date                                       2025-12-13 17:00:00 
motion        As Victor Frankenstein we would accept the off...
score                                               vyhráli 3:0
score_upd                                           vyhráli 3:0
judges                                                      NaN
comp                                           Debate Cup XXIX.
league              {'name': 'Debate League XXXI.', 'id': '47'}
tournament    {'name': 'Druhý turnaj Debate League 2025/2026...
teams         [{'team_name': 'Bít, či nebýt', 'side': 'aff',...
Name: 6, dtype: object

In [38]:
d = pd.read_csv("debate_data.csv")
d.head()

,type,id,date,comp,league_name,league_id,motion,tournament_name,tournament_id,judges_scoring,score,teams
0,debate,11012,2025-04-28 09:15:00,Debates For Fun,NaN,NaN,Economic globalization has brought more harm t...,NaN,NaN,NaN,vyhráli 2:1,"[{'team_name': 'G-session', 'side': 'aff', 'sp..."
1,debate,11013,2025-04-28 10:30:00,Debaty pro radost,NaN,NaN,Společenská hnutí bez prominentního vůdce jsou...,NaN,NaN,NaN,vyhráli 3:0,"[{'team_name': 'The Economist', 'side': 'aff',..."
2,debate,11014,2025-05-06 14:50:00,Debatní pohár XXIX.,NaN,NaN,Jedničkáři by měli mít právo nechodit do školy.,NaN,NaN,"[{'name': 'Hon Filip', 'side': 'neg', 'score':...",vyhráli 3:0,"[{'team_name': 'Oxid hořečnatý 1', 'side': 'af..."
3,debate,11015,2025-05-05 17:15:00,Debaty pro radost,NaN,NaN,Organizované náboženství přináší více škody ne...,NaN,NaN,"[{'name': 'Soukup Václav', 'side': 'neg', 'sco...",vyhráli 3:0,"[{'team_name': 'OGL - Pozitivní Krysičky', 'si..."
4,debate,11016,2025-05-13 09:15:00,Debatní pohár XXIX.,NaN,NaN,Každý občan žijící pod totalitním režimem má m...,Stojanův pohár 2025,314.0,"[{'name': 'Kotůlková Renáta', 'side': 'aff', '...",vyhráli 3:0,"[{'team_name': 'Fireflies', 'side': 'aff', 'sp..."


In [50]:
d.tournament_name.value_counts().sum()

np.int64(297)

In [57]:
type(d.judges_scoring[5])

str

In [59]:
dd = pd.read_csv("debate_data.csv")
dd.head()

,type,id,date,comp,league_name,league_id,motion,tournament_name,tournament_id,judges_scoring,score,teams
0,debate,10700,2025-01-26 09:31:00,Debatní pohár XXVIII.,Debatní liga XXX.,44.0,Rozvinuté země by měly kompenzovat rozvojovým ...,Druhý turnaj Debatní ligy 2024/2025,307.0,"[{'name': 'Kalouda Dominik', 'side': 'neg', 's...",vyhráli 3:0,"[{'team_name': 'Výprodej akustických kytar', '..."
1,debate,10701,2025-01-26 09:31:00,Debatní pohár XXVIII.,Debatní liga XXX.,44.0,Rozvinuté země by měly kompenzovat rozvojovým ...,Druhý turnaj Debatní ligy 2024/2025,307.0,"[{'name': 'Navrátilová Anežka', 'side': 'neg',...",vyhráli 3:0,"[{'team_name': 'Fretky Alfredky', 'side': 'aff..."
2,debate,10702,2025-01-26 09:31:00,Debatní pohár XXVIII.,Debatní liga XXX.,44.0,Rozvinuté země by měly kompenzovat rozvojovým ...,Druhý turnaj Debatní ligy 2024/2025,307.0,"[{'name': 'Kotůlková Renáta', 'side': 'neg', '...",vyhráli 3:0,"[{'team_name': 'Rohlík', 'side': 'aff', 'speak..."
3,debate,10703,2025-01-26 09:31:00,Debatní pohár XXVIII.,Debatní liga XXX.,44.0,Rozvinuté země by měly kompenzovat rozvojovým ...,Druhý turnaj Debatní ligy 2024/2025,307.0,"[{'name': 'Neužil Jakub', 'side': 'neg', 'scor...",vyhráli 3:0,"[{'team_name': 'gleep glorp', 'side': 'aff', '..."
4,debate,10704,2025-01-26 09:31:00,Debatní pohár XXVIII.,Debatní liga XXX.,44.0,Rozvinuté země by měly kompenzovat rozvojovým ...,Druhý turnaj Debatní ligy 2024/2025,307.0,"[{'name': 'Šotolová Marta', 'side': 'aff', 'sc...",vyhráli 2:1,"[{'team_name': 'Louboutins', 'side': 'aff', 's..."


In [60]:
dd.shape

(680, 12)

In [62]:
dd.isna().sum()

type                 0
id                   0
date                 3
comp                 3
league_name        298
league_id          298
motion               3
tournament_name    115
tournament_id      115
judges_scoring      10
score                9
teams                3
dtype: int64

In [68]:
dd["judges_scoring"][440]

"[{'name': 'Bojko David', 'side': 'neg', 'score': '3:0'}]"